# **DSFM Project**: Pre-Screen Funding Applications

Creator: [Data Science for Managers - EPFL Program](https://www.dsfm.ch)  
Source:  [https://github.com/dsfm-org/code-bank.git](https://github.com/dsfm-org/code-bank.git)  
License: [MIT License](https://opensource.org/licenses/MIT). See open source [license](LICENSE) in the Code Bank repository. 

-------------

## Overview

Founded in 2000 by a high school teacher in the Bronx, DonorsChoose.org empowers public school teachers from across the country to request much-needed materials and experiences for their students. At any given time, there are thousands of classroom requests that can be brought to life with a gift of any amount.

DonorsChoose.org receives hundreds of thousands of project proposals each year for classroom projects in need of funding. Right now, a large number of volunteers is needed to manually screen each submission before it's approved to be posted on the DonorsChoose.org website.

In this project, we will predict the probability that a DonorsChoose.org project proposal submitted by a teacher will be approved, using the text of project descriptions as well as additional metadata about the project, teacher, and school. DonorsChoose.org can then use this information to identify projects most likely to need further review before approval. Hence, the organisation can save the time of volunteer reviewers so that they can focus on more promising projects.

Your machine learning algorithm can help more teachers get funded more quickly, and with less cost to DonorsChoose.org, allowing them to channel even more funding directly to classrooms across the country.

<img src="https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg" width="500" height="500" align="center"/>

Image source: https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg

As you will see, this dataset includes many different kinds of features with structured and unstructured data. You need to predict whether an application needs further study. To assess the quality of your predictions, consider the area under the curve (AUC) performance metric.

The dataset consists of application materials (see *application_data.csv*) and resources requested (see *resource_data.csv*).

The application materials data (see *application_data.csv*) contains the following features.

| Feature name  | Description  |
|----------------|--------------|
| id  | Unique id of the project application    |
| teacher_id    | id of the teacher submitting the application  |
| teacher_prefix    | title of the teacher's name (Ms., Mr., etc.)    |
| school_state    | US state of the teacher's school    |
| project_submitted_datetime    | application submission timestamp    |
| project_grade_category    | school grade levels (PreK-2, 3-5, 6-8, and 9-12)   |
| project_subject_categories   | category of the project (e.g., "Music & The Arts")    |
| project_subject_subcategories    | sub-category of the project (e.g., "Visual Arts")    |
| project_title    | title of the project    |
| project_essay_1    | first essay*   |
| project_essay_2    | second essay*    |
| project_essay_3    | third essay*   |
| project_essay_4    | fourth essay*  |
| project_resource_summary    | summary of the resources needed for the project    |
| teacher_number_of_previously_posted_projects   | number of previously posted applications by the submitting teacher    |
| project_is_approved    | whether DonorsChoose proposal was accepted (0="rejected", 1="accepted"); train.csv only    |


\*Note: Prior to May 17, 2016, the prompts for the essays were as follows:

  * project_essay_1: "Introduce us to your classroom"  

  * project_essay_2: "Tell us more about your students"  

  * project_essay_3: "Describe how your students will use the materials you're requesting"  

  * project_essay_4: "Close by sharing why your project will make a difference"  

Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the prompts for the first 2 essays were changed to the following:

  * project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."  

  * project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"  

For all projects with project_submitted_datetime of 2016-05-17 and later, the values of project_essay_3 and project_essay_4 will be missing (i.e. NaN).


Proposals also include resources requested. Each project may include multiple requested resources (see *resource_data.csv*). Each row in *resource_data.csv* corresponds to a resource, so multiple rows may tie to the same project by id.

The resource data contains the following features.

| Feature name  | Description  |
|----------------|--------------|
| id  | unique id of the project application; joins with test.csv. and train.csv on id   |
| description    |  description of the resource requested     |
| quantity   | quantity of resource requested      |
| price    | price of resource requested      |


Dataset source: [Kaggle DonorsChoose.org Application Screening challenge](https://www.kaggle.com/c/donorschoose-application-screening/data)

The problem is divided into several parts. For each part, you will have time to work on the question yourself. Feel free to go back to the Demo, use Google/Stackoverflow and work with your neighbour. Together, we will review and discuss the solution to each part.

-------------

## **Part 0**: Setup

In [ ]:
# Import all packages 
import re
import nltk
nltk.download('stopwords')

# Use short-hand for standard packages
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import texthero          as hero

# Import individual functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition           import PCA
from sklearn.model_selection         import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.pipeline                import Pipeline
from scipy.sparse                    import hstack
from bs4                             import BeautifulSoup
from nltk.corpus                     import stopwords 
from nltk.stem                       import SnowballStemmer
from xgboost                         import XGBClassifier
from collections                     import Counter

# Special code to ignore un-important warnings 
import warnings
warnings.filterwarnings('ignore')

# Ensure that output of plotting commands is displayed inline
%matplotlib inline

# Set the maximum number of rows displayed 
pd.options.display.max_rows = 1000

In [ ]:
# Define all constants

SEED    = 41  # base to generate a random number
SCORE   = 'roc_auc'
FIGSIZE = (16, 10)

## **Part 1**: Data Preprocessing and EDA

First, we have to download the data. Next, we would like to understand the main characteristics of the dataset. We might need to transform and clean some features before we can specify a statistical model.

**Q 1:** Unlike other projects, this project includes a training set too big for GitHub. Through the terminal lab of Jupyter lab, download the data using the *wget* command, unzip it using the *zip* command and check that it's in the root directory of the project. 

Locations : 

    Applications dataset: https://storage.googleapis.com/dsfm-datasets/text-applications/application_data.csv.zip
    Resources dataset: https://storage.googleapis.com/dsfm-datasets/text-applications/resource_data.csv.zip

Hint: Use *wget* and *unzip* commands. Use *!* followed by a bash command in a cell to run a bash command.

In [ ]:
!wget -N 'https://storage.googleapis.com/dsfm-datasets/text-applications/application_data.csv.zip'
!wget -N 'https://storage.googleapis.com/dsfm-datasets/text-applications/resource_data.csv.zip'
!unzip -o application_data.csv.zip
!unzip -o resource_data.csv.zip

**Q 2:** Load the two datasets and investigate their features. What could be a unifying strategy to create the same "project essay" columns? Do the essays share common topics? Implement your strategy. We will deal with missing values afterwards.

In [ ]:
# Load applications
applications = pd.read_csv('application_data.csv')

print('Application dataset')
print(applications.shape)
applications.head()


In [ ]:
# Load resources
resources = pd.read_csv('resource_data.csv')

print('Resources dataset')
print(resources.shape)
resources.head()


In [ ]:
# Create two new features to represent essays: one about the class environment, the other about the project

print('Shape before: {}'.format(applications.shape))

# Fill new environment column
project_essay_env = []

for i, row in applications.iterrows():
    
    if pd.isna(row["project_essay_3"]) and pd.isna(row["project_essay_4"]):
        project_essay_env.append(row['project_essay_1'])
        
    else:
        project_essay_env.append(str(row["project_essay_1"]) + " " + str(row["project_essay_2"]))

assert len(project_essay_env) == applications.shape[0], 'New column length does not match'
applications['project_essay_env'] = project_essay_env

# Fill new project column
project_essay_proj = []

for i, row in applications.iterrows():
    
    if pd.isna(row["project_essay_3"]) and pd.isna(row["project_essay_4"]):
        project_essay_proj.append(row['project_essay_2'])
        
    else:
        project_essay_proj.append(str(row["project_essay_3"]) + " " + str(row["project_essay_4"]))

assert len(project_essay_proj) == applications.shape[0], 'New column length does not match'
applications['project_essay_proj'] = project_essay_proj

print('Shape after: {}'.format(applications.shape))

In [ ]:
# Drop redundant essay features
applications.drop(columns=['project_essay_1', 'project_essay_2', 'project_essay_3', 'project_essay_4'], inplace=True)

# Drop rows with missing values
applications.dropna(axis=0, inplace=True)

# Check if there is still any missing value
print('Shape: {}\n'.format(applications.shape))
print(applications.isnull().sum())

**Q 3:** Merge the resources and application datasets on the *id* feature. You can aggregate the total cost of requested resources for every project and merge it into applications dataset. 

Hint: Use *groupby* and *agg* functions of dataframe to create aggregates. Use *merge* function of *Pandas* to merge two dataframes.

In [ ]:
# Create useful feature out of resources: total requested budget
resources['total_budget'] = resources['quantity'] * resources['price']

# Drop non-informative features
resources.drop(columns=['quantity','price', 'description'], inplace=True)

# Aggregate total budget by id
resources_agg = resources.groupby('id', as_index=False).agg({"total_budget": "sum"})
print('Resources shape: {}'.format(resources_agg.shape))

# CODE HERE 
# Merge two datasets
applications_full = None

# Assert OK to proceed 
assert applications_full is not None, 'HINT: you need to complete the code to proceed.'

# Check data type of new datasets
print('Merged shape: {}\n'.format(applications_full.shape))
applications_full.dtypes

**Q 4:**  Separate your merged dataset into features and target variables and visualize the distribution of the target. Why is area under the ROC curve (AUC) a suitable classification metric? Why can we not use accuracy?

In [ ]:
# Split target and feature
target   = applications_full['project_is_approved']
features = applications_full.drop(columns=['project_is_approved','id', 'project_submitted_datetime'])

print('Target: {}'.format(target.shape))
print('Features: {}'.format(features.shape))

# Plot histogram of target
target.hist()

## **Part 2**: Encode Categories

**Q 1:** What would be a possible issue with one-hot encoding of teacher_id column? Should we keep this feature? There might be another, more useful teacher-specific feature we can use. Which one?

In [ ]:
features.drop(columns = ['teacher_id'], inplace = True)
features.shape

**Q 2:** Encode these categorical features using one-hot-encoding: ['teacher_prefix', 'school_state', 'project_grade_category']

In [ ]:
cols_to_transform = ['teacher_prefix', 'school_state', 'project_grade_category']

features = pd.get_dummies(data = features, columns = cols_to_transform, drop_first=True)
features.shape

**Q 3:** What could be the issue with simply one-hot-encoding of *project_subject_category* and *project_subject_subcategory* features? Come up with a sensible approach and implement it.

In [ ]:
# Let's start with the subject categories

# Extract only the categories column
features_categories = features[['project_subject_categories']].copy()
features_categories['project_subject_categories_list'] = features_categories['project_subject_categories'].str.split(', ')
features_categories.drop(columns=['project_subject_categories'], inplace=True)

# Explode the column
features_categories = features_categories.explode('project_subject_categories_list')
print('Number of categories: {}'.format(len(set(features_categories['project_subject_categories_list'].values))))
Counter(features_categories['project_subject_categories_list'].values)

In [ ]:
# One-hot encode the applications now and group them by index values
features_categories = pd.get_dummies(features_categories, columns=['project_subject_categories_list'])
features_categories = features_categories.groupby(features_categories.index).sum()
features_categories.shape

In [ ]:
# Now repeat the process for subject subcategories

# Extract only the categories column
features_subcategories = features[['project_subject_subcategories']].copy()
features_subcategories['project_subject_subcategories_list'] = features_subcategories['project_subject_subcategories'].str.split(', ')
features_subcategories.drop(columns=['project_subject_subcategories'], inplace=True)

# Explode the column
features_subcategories = features_subcategories.explode('project_subject_subcategories_list')
print('Number of sub-categories: {}'.format(len(set(features_subcategories['project_subject_subcategories_list'].values))))
Counter(features_subcategories['project_subject_subcategories_list'].values)


In [ ]:
# One-hot encode the applications now and group them by index values
features_subcategories = pd.get_dummies(features_subcategories, columns=['project_subject_subcategories_list'])
features_subcategories = features_subcategories.groupby(features_subcategories.index).sum()
features_subcategories.shape

In [ ]:
# Merge one-hot encoded categories and sub-categories with the dataframe

# Categories
features = features.merge(features_categories, how='left', left_index=True, right_index=True)

# CODE HERE
# Concatenate sub-category dummies into existing features data frame
features = None

# Assert OK to proceed 
assert features is not None, 'HINT: you need to complete the code to proceed.'

print(features.shape)
features.head()

In [ ]:
# Drop the original columns
features.drop(columns=['project_subject_categories','project_subject_subcategories'], inplace=True)

features.shape

## **Part 3**: Encode Text

In this part, we use a new data science package called [TextHero](https://github.com/jbesomi/texthero) for easy text pre-processing.

In [ ]:
# Concat all textual features
features['app_text'] = features['project_title'] + ' ' + features['project_essay_env'] + ' ' + features['project_essay_proj'] + ' ' + features['project_resource_summary']
features.drop(columns = ['project_title',
                         'project_resource_summary',
                         'project_essay_env',
                         'project_essay_proj'], inplace = True)


features['app_text_clean'] = features['app_text'].pipe(hero.clean)
features_pca = features['app_text_clean'].pipe(hero.tfidf, max_features=500).pipe(hero.pca, n_components=50)
textual_features_pca = pd.DataFrame(dict(zip(features_pca.index, features_pca.values))).T

**Q 4:** You can think of generating some new features out of text which are beneficial for the target application in mind. In particular, add the number of words in application text as a new feature.


In [ ]:
# CODE HERE
# Generate 'number of words' feature
num_words = None

# Assert OK to proceed 
assert num_words is not None, 'HINT: you need to complete the code to proceed.'

features['num_words'] = num_words

## **Part 4**: Prepare Features and Baseline

**Q 1:** Organize your features into three dataframes of *textual_features* (projected to 50 PCA dimensions), *non_textual_features* and *all_features*. Check the data types of all features: are they all numeric? 

In [ ]:
# Textual features
textual_features_pca.columns = ['dim_' + str(col) for col in textual_features_pca.columns]
textual_features = textual_features_pca.copy()

# CODE HERE
# Only keep non-textual featues, i.e. drop 2 application text columns
non_textual_features = None

# Assert OK to proceed 
assert non_textual_features is not None, 'HINT: you need to complete the code to proceed.'

# All features: are all types numeric? 
all_features = pd.concat([non_textual_features, textual_features], axis = 1)
all_features.dtypes

**Q 2:** Come up with a conceptual baseline to compare with your models. No coding required.

## **Part 5**: Gradient Boosting Trees

**Q 1:** Fit a gradient boosting tree model from the the *XGBoost* library to your data and tune the value of the *n_estimators* parameter, the number of trees to fit. Use nested cross-validation function with 3 splits each in the inner and outer folds. Assess the performance of your model when tuning the *n_estimators* parameter and using the following sets of features:

1. Non-textual features
2. Textual features
3. All features

Hint: Use the following function for nested cross-validation (Try to understand it before using it):

    from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score

    def nested_cv(X, y, est_pipe, p_grid, p_score, n_splits_inner = 3, n_splits_outer = 3, n_cores = 1, seed = 0):

        # cross-validation schema for inner and outer loops
        inner_cv = StratifiedKFold(n_splits = n_splits_inner, shuffle = True, random_state = seed)
        outer_cv = StratifiedKFold(n_splits = n_splits_outer, shuffle = True, random_state = seed)

        # grid search to tune hyper parameters
        est = GridSearchCV(estimator = est_pipe, param_grid = p_grid, cv = inner_cv, scoring = p_score, n_jobs = n_cores)

        # nested CV with parameter optimization
        nested_scores = cross_val_score(estimator = est, X = X, y = y, cv = outer_cv, scoring = p_score, n_jobs = n_cores)

        print('Average score: %0.4f (+/- %0.4f)' % (nested_scores.mean(), nested_scores.std() * 1.96))
        
Moreover use the *XGBClassifier* class from *xgboost* package, which has a similar interface to other sklearn classifiers. *XGboost* library includes high perfromance implementations of gradient boosting trees. 

In [ ]:
# Nested cross validation function

def nested_cv(X, y, est_pipe, p_grid, p_score, n_splits_inner = 3, n_splits_outer = 3, n_cores = 1, seed = SEED):

    # Cross-validation schema for inner and outer loops
    inner_cv = StratifiedKFold(n_splits = n_splits_inner, shuffle = True, random_state = seed)
    outer_cv = StratifiedKFold(n_splits = n_splits_outer, shuffle = True, random_state = seed)
    
    # Grid search to tune hyper parameters
    est = GridSearchCV(estimator = est_pipe, param_grid = p_grid, cv = inner_cv, scoring = p_score, n_jobs = n_cores)

    # Nested CV with parameter optimization
    nested_scores = cross_val_score(estimator = est, X = X, y = y, cv = outer_cv, scoring = p_score, n_jobs = n_cores)
    
    print('Average score: %0.4f (+/- %0.4f)' % (nested_scores.mean(), nested_scores.std() * 1.96))

# Define pipeline
estimators = []
estimators.append(('xgb_clf', XGBClassifier()))
xgb_pipe = Pipeline(estimators)
xgb_pipe.set_params(xgb_clf__n_jobs = -1)
xgb_pipe.set_params(xgb_clf__random_state = SEED)

# Setup possible values of parameters to optimize over
p_grid = {"xgb_clf__n_estimators": [int(i) for i in np.linspace(10, 100, 10)]}

In [ ]:
%%time

# Non textual features
nested_cv(X = non_textual_features, y = target, est_pipe = xgb_pipe, p_grid = p_grid, p_score = SCORE, n_cores = -1)

In [ ]:
%%time

# CODER HERE
# Textual features (copy what you ran in the cell just above, but with textual features only)
None

In [ ]:
%%time

# All features
nested_cv(X = all_features, y = target, est_pipe = xgb_pipe, p_grid = p_grid, p_score = SCORE, n_cores = -1)

**Q 2:** Print a list of the 10 most important features considering all features as pedictors. 

In [ ]:
# Extract feature importances

xgb_clf = xgb_pipe.named_steps['xgb_clf']
xgb_clf.fit(all_features, target)

i = 0

for index in reversed(np.argsort(xgb_clf.feature_importances_)):
    
    print(all_features.columns[index].ljust(50) , ':', xgb_clf.feature_importances_[index])
    i = i + 1

    if (i == 10):
        break

## **Bonus**: Further Reading

- Further ideas for feature engineering: https://www.kaggle.com/shivamb/extensive-text-data-feature-engineering
- Winning submission on Kaggle (don't worry if you don't understand everything): https://www.kaggle.com/shadowwarrior/1st-place-solution